In [1]:
import pathlib
import numpy

from bluepyefe.extract import read_recordings, extract_efeatures_at_targets, compute_rheobase,\
    mean_efeatures, create_feature_protocol_files
from bluepyefe.plotting import plot_all_recordings_efeatures

from reader import wcp_reader
from targets import targets, ecode_to_index, ecodes_timings

In [2]:
def build_metadata(cell_id="cell1", ephys_dir="../../ephys_data/210301/", repetition_as_different_cells=True):
    
    files_metadata  = {}
    
    if not(repetition_as_different_cells):
        files_metadata[cell_id] = {}

    for repetition in range(2, 5):
        
        if repetition_as_different_cells:
            current_id = f"{cell_id}_rep{repetition}"
            files_metadata[current_id] = {}
        else:
            current_id = cell_id
            
        for ecode in ecode_to_index:

            file_path = pathlib.Path(ephys_dir) / f"{cell_id}_run{repetition}.{ecode_to_index[ecode]}.wcp"
            
            if not file_path.is_file():
                print(f"Missing trace {file_path}")
                continue
            
            metadata= {
                "filepath": str(file_path),
                "i_unit": "pA",
                "t_unit": "s",
                "v_unit": "mV",
                "ljp": 14.
            }

            metadata.update(ecodes_timings[ecode])

            if ecode not in files_metadata[current_id]:
                files_metadata[current_id][ecode] = [metadata]
            else:
                files_metadata[current_id][ecode].append(metadata)

    return files_metadata

In [3]:
output_directory = "./efeatures/"

cells = read_recordings(
    files_metadata=build_metadata(),
    recording_reader=wcp_reader
)

/gpfs/bbp.cscs.ch/home/damart/venv_lfpy/lib/python3.8/site-packages/neo/io/basefromrawio.py:107: UserWarning: default "signal_group_mode" have change in version 0.9:now all channels are group together in AnalogSignal
  warnings.warn('default "signal_group_mode" have change in version 0.9:'


In [4]:
extract_efeatures_at_targets(
    cells, 
    targets,
    threshold=-20.
)

/gpfs/bbp.cscs.ch/project/proj38/home/damart/LFPy/BluePyEfe/bluepyefe/cell.py:152: RuntimeWarning: Mean of empty slice
  recording.efeatures[efeature] = numpy.nanmean(f)


In [5]:
compute_rheobase(
    cells, 
    protocols_rheobase=['IDthres']
)

In [6]:
protocols = mean_efeatures(cells, targets, use_global_rheobase=True)

Could not compute average ecode for protocol HyperDepol target -120 because it didn't match any recordings
Could not compute average ecode for protocol sAHP target 250 because it didn't match any recordings
Could not compute average ecode for protocol PosCheops target 300 because it didn't match any recordings


In [7]:
efeatures, protocol_definitions, current = create_feature_protocol_files(
    cells,
    protocols,
    output_directory=output_directory,
    threshold_nvalue_save=1,
    write_files=True,
)

Standard deviation for efeatures mean_frequency for stimulus firepattern_200 is 0. and will be set to 1e-3
Standard deviation for efeatures burst_number for stimulus firepattern_200 is 0. and will be set to 1e-3
Standard deviation for efeatures adaptation_index2 for stimulus firepattern_200 is 0. and will be set to 1e-3
Standard deviation for efeatures ISI_CV for stimulus firepattern_200 is 0. and will be set to 1e-3
Standard deviation for efeatures ISI_log_slope for stimulus firepattern_200 is 0. and will be set to 1e-3
Standard deviation for efeatures inv_time_to_first_spike for stimulus firepattern_200 is 0. and will be set to 1e-3
Standard deviation for efeatures inv_first_ISI for stimulus firepattern_200 is 0. and will be set to 1e-3
Standard deviation for efeatures inv_second_ISI for stimulus firepattern_200 is 0. and will be set to 1e-3
Standard deviation for efeatures inv_third_ISI for stimulus firepattern_200 is 0. and will be set to 1e-3
Standard deviation for efeatures inv_f

In [ ]:
plot_all_recordings_efeatures(
    cells, protocols, output_dir=output_directory
)

/gpfs/bbp.cscs.ch/project/proj38/home/damart/LFPy/BluePyEfe/bluepyefe/protocol.py:62: RuntimeWarning: Mean of empty slice
  return numpy.nanmean(list_feature), numpy.nanstd(list_feature)
/gpfs/bbp.cscs.ch/home/damart/venv_lfpy/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/gpfs/bbp.cscs.ch/home/damart/venv_lfpy/lib/python3.8/site-packages/numpy/core/_asarray.py:102: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)
/gpfs/bbp.cscs.ch/project/proj38/home/damart/LFPy/BluePyEfe/bluepyefe/protocol.py:62: RuntimeWarning: Mean of empty slice
  return numpy.nanmean(list_feature), numpy.nanstd(list_feature)
/gpfs/bbp.cscs.ch/home/damart/venv_lfpy/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=dd

/gpfs/bbp.cscs.ch/home/damart/venv_lfpy/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/gpfs/bbp.cscs.ch/home/damart/venv_lfpy/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/gpfs/bbp.cscs.ch/project/proj38/home/damart/LFPy/BluePyEfe/bluepyefe/protocol.py:62: RuntimeWarning: Mean of empty slice
  return numpy.nanmean(list_feature), numpy.nanstd(list_feature)
/gpfs/bbp.cscs.ch/home/damart/venv_lfpy/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/gpfs/bbp.cscs.ch/home/damart/venv_lfpy/lib/python3.8/site-packages/numpy/core/_asarray.py:102: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)
/gpfs/bbp.cscs.